In [1]:
from dotenv import load_dotenv
import os
from circuit_tracer import ReplacementModel, attribute
import torch
from pathlib import Path

load_dotenv()

False

In [2]:
model_name = 'google/gemma-2-2b'
transcoder_name = "gemma"
model = ReplacementModel.from_pretrained(model_name, transcoder_name, dtype=torch.bfloat16)

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2-2b into HookedTransformer


In [3]:
graph_dir = 'graphs'
graph_name = 'ngsa.pt'
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

In [ ]:
prompt = "Non-dominated Sorting Genetic Algorithm II (N"  # What you want to get the graph for
max_n_logits = 10   # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 8192  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size=256  # Batch size when attributing
offload=None # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report

graph = attribute(
    prompt=prompt,
    model=model,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=offload,
    verbose=verbose
)

graph.to_pt(graph_path)

Phase 0: Precomputing activations and vectors
Precomputation completed in 0.19s
Found 10633 active features
Phase 1: Running forward pass
Forward pass completed in 0.70s
Phase 2: Building input vectors
Selected 7 logits with cumulative probability 0.9531
Will include 8192 of 10633 feature nodes
Input vectors built in 0.48s
Phase 3: Computing logit attributions
Logit attributions completed in 0.19s
Phase 4: Computing feature attributions
Feature influence computation: 100%|██████████| 8192/8192 [00:06<00:00, 1303.26it/s]
Feature attributions completed in 6.29s
Attribution completed in 8.03s


: 